# Evaluation

In [ ]:
import time
import numpy as np
import pandas as pd
from pathlib import Path
from collections import OrderedDict
from tools import eval_methods, selection_methods
from tqdm.auto import tqdm
tqdm.pandas()

### Evaluate candidate selection and ranking

In [ ]:
approach = "partial_match"
num_candidates = 3
setting = "test"
exact_station = True

test_df = pd.read_pickle("../processed/resolution/candranking_" + approach + "_" + setting + str(num_candidates) + ".pkl")

combinations = [["stations"], ["stations", "alts"], ["stations", "places", "alts"]]
candrank_approaches = [x.replace("cr_", "").replace("_stations", "") for x in test_df if x.startswith("cr_") and x.endswith("stations")]

eval_results = []
for combination in combinations:
    for approach in candrank_approaches:
        
        if "places" in combination:
            exact_station = False
        
        # Get relevant columns from dataframe:
        relv_columns = []
        for c in combination:
            relv_columns.append("cr_" + approach + "_" + c)
        
        # Report performance:
        p = test_df.apply(lambda row: eval_methods.pAt(row, approach, relv_columns, exact_station), axis=1).mean()
        mapAt = test_df.apply(lambda row: eval_methods.avgP(row, approach, relv_columns, exact_station), axis=1).mean()
        isRetrieved = test_df.apply(lambda row: eval_methods.isRetrieved(row, approach, relv_columns, exact_station), axis=1).mean()
        eval_results.append([approach + ": " + "+".join(combination), p, mapAt, isRetrieved])
        
annotation = "strict" if exact_station else "appr"
cr_eval_df = pd.DataFrame(eval_results, columns = ["Approach:" + annotation, "p@" + str(num_candidates), "map@" + str(num_candidates), "retr@" + str(num_candidates)])
cr_eval_df.round(3)
print(cr_eval_df.round(2).to_latex(index=False))